# Intro to AI Agents

> *This notebook should work well with the **`conda_python3`** kernel in SageMaker Studio on ml.t3.medium instance*

---

In this notebook, we will introduce the concept of AI agents and how they can be used to solve problems. Unlike static LLM workflows, AI agents are dynamic and can adapt to a much wider range of problems. They are particularly well suited when we don't know the exact parameters of each user interaction. Some cases which may call for AI agents include:
- Goal oriented tasks such as researching a topic, analyzing data across many sources, or helping find the best product for a user.
- Tasks that may involve a wide variety of user inputs, such as document processing where there may be a wide variety of document formats and data that needs to be extracted.
- Tasks that may involve usage of various external tools or APIs, without a fixed sequential order of operations.

There are numerous frameworks available for implementing agents including [Amazon Bedrock](https://aws.amazon.com/bedrock/agents/), [CrewAI](https://www.crewai.com/), and [LangGraph](https://www.langchain.com/langgraph). The frameworks vary in their capabilities and complexity, but all provide a way to define and deploy agents that can interact with users and other systems.For this notebook, we will use a lightweight framework from Hugging Face called [smolagents](https://github.com/huggingface/smolagents). This framework provides a very simple and lightweight way to define agents that can interact with users and other systems, and is thus well suitable for quick prototyping and experimentation.


---

In [2]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

Validating base environment
Base environment validated successfully


Validating lab environment from requirements.txt ✨

ENVIRONMENT STATUS
✅  boto3==1.37.9 is installed
✅  botocore==1.37.9 is installed
✅  smolagents==1.9.2 is installed
✅  litellm==1.61.15 is installed
✅  yfinance>=0.2.54 is installed
✅  pandas-datareader is installed

All required libraries are installed.🎉
You may proceed with the lab! 🚀

> 🚨 **Caution** You may get an exception running the cell bellow. If that's the case, please restart the kernel by clicking **Kernell** -> **Restart Kernel**. Alternatively click the refresh icon on the notebook toolbar above

In [3]:
required_models = [
    "amazon.titan-embed-text-v1",
    "us.anthropic.claude-3-5-haiku-20241022-v1:0",
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    "us.amazon.nova-pro-v1:0",
]
validate_model_access(required_models)

MODEL ACCESS STATUS
✅  amazon.titan-embed-text-v1 is accessible
✅  us.anthropic.claude-3-5-haiku-20241022-v1:0 is accessible
✅  us.anthropic.claude-3-5-sonnet-20241022-v2:0 is accessible
✅  us.amazon.nova-pro-v1:0 is accessible

All required models are accessible.🎉
You may proceed with the lab! 🚀

## Simple Search Agent

Let's build a simple search agent that can search for information on the web. The agent will take a query from the user and return the top search results. We will use DuckDuckGo as the search engine for this agent.
smolagents offers two types of agents:
- [CodeAgent](https://huggingface.co/papers/2402.01030): Invokes tools via generated python code snippets. This provides flexibility in terms of how the agent interacts with external tools as it's able to expand on the capabilities of the tools by incorporating custom code.
- [ToolCallingAgent](https://huggingface.co/learn/agents-course/en/unit2/smolagents/tool_calling_agents): Invokes tools by generating a JSON output that contains the tool name and parameters and the invocation parameters. This approach is more rigid as the model is limited to the capabilities of the tools it calls however it may prove to be more efficient and secure in some cases as it does not involve running arbitrary code.

As as an example if we have a tool that can reterieve stock data for a given stock symbol. If the agent receives a task that involves retrieving stock data for say "AAPL", "MSFT" and "GOOGL", the CodeAgent will write a code snippet with a simple for loop that will retrieve the stock data for each symbol. While the ToolCallingAgent will generate a JSON output that contains the tool name and parameters and the invocation parameters for each stock symbol. Additionally, if the tasks requires invoking additional tools to analyze the stock data, the CodeAgent can potentially tackle this in a single code snippet, while the ToolCallingAgent will need to make multiple calls to the tools which could be less efficient.

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, tool
from typing import List
import pandas as pd
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import numpy as np


In [5]:
MODEL_ID = "bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0"
model = LiteLLMModel(model_id=MODEL_ID, temperature=0)

In [6]:
# define as simple agent that has access to online
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)
agent.run("How many years would it take an average person to watch all of the content on Amazon prime video?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many years would it take an average person to watch all of the content on Amazon prime video?               │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0 ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  web_search(query="How many hours of content are on Amazon Prime Video total library size")                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'web_search(query="How many hours of content are on Amazon Prime Video total library 
size")' due to: DuckDuckGoSearchException: https://html.duckduckgo.com/html 202 Ratelimit

[Step 0: Duration 6.38 seconds| Input tokens: 2,373 | Output tokens: 97]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Rounding the result for clarity                                                                                
  years_to_watch_rounded = round(43.15, 2)                                                                         
  final_answer(f"{years_to_watch_rounded} years")                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 43.15 years

[Step 4: Duration 7.10 seconds| Input tokens: 14,905 | Output tokens: 696]

'43.15 years'

## Stock Analysis Agent
Let's look at a more complex example. We will build an agent that can analyze stock data. In addition to web search, this agent will have access to the following tools:
- `get_ticker_data`: This tool will retrieve stock data for a given stock symbol.
- `get_fred_data`: This tool will retrieve economic data from the Federal Reserve Economic Data (FRED) API.
- `run_ols_regression`: This tool will run an ordinary least squares regression which can be used to analyze the relationship between two variables

Defining tools is easy as we merely need to decorate a function with the `@tool` decorator. The function should contain a docstring in [Google-style](https://sphinxcontrib-napoleon.readthedocs.io/en/latest/example_google.html) format that describes the tool's inputs and outputs. Providing clear documentation is important as it informs the agent about the tool's capabilities and how to use it.

In [7]:
@tool
def get_ticker_data(
    tickers: List[str],
    start_date: str,
    end_date: str,
    metric: str = "all",
    sampling: str = "monthly",
) -> dict:
    """Downloads historical stock data from Yahoo Finance and returns it as a dictionary.

    Examples:
        >>> get_ticker_data(["AAPL"], "2023-01-01", "2023-12-31", "Close", "weekly")
        {"AAPL": [{"Date": "2023-01-06", "Close": 129.619995}, {"Date": "2023-01-13", "Close": 134.759995}, ...]}

        >>> get_ticker_data(["AAPL", "MSFT"], "2023-01-01", "2023-12-31", "all", "monthly")
        {"AAPL": [{"Date": "2023-01-31", "Open": 144.479996, "High": 147.229996, "Low": 141.320007, "Close": 144.289993, "Adj Close": 143.839996, "Volume": 77663600}, ...],
          "MSFT": [{"Date": "2023-01-31", "Open": 250.089996, "High": 256.25, "Low": 242.529999, "Close": 252.509995, "Adj Close": 251.873795, "Volume": 47146900}, ...]}

    Args:
        tickers: A list of stock ticker symbols (e.g., ["AAPL", "MSFT"]).
        start_date: The start date for the data (e.g., "2023-01-01").
        end_date: The end date for the data in YYYY-MM-DD format (e.g., "2023-12-31").
        metric:  If "all", returns all available data columns (Open, High, Low, Close, Volume).
            Otherwise, specifies a single metric to return (e.g., "Close"). Defaults to "all".
        sampling: The frequency of the data. Can be "daily", "weekly", or "monthly". Defaults to "monthly".

    Returns:
        dict: A dictionary where keys are ticker symbols and values are lists of historical data records.
             Each record is a dictionary containing 'Date' and the requested metrics.

    Raises:
        ValueError: If an invalid sampling frequency is provided.


    """

    df = yf.download(tickers, start=start_date, end=end_date)

    if metric != "all":
        df = df[metric]

    if sampling == "weekly":
        df = df.resample("W-SAT").last()
    elif sampling == "monthly":
        df = df.resample("ME").last()
    elif sampling == "quarterly":
        df = df.resample("QE").last()
    elif sampling == "daily":
        pass
    else:
        raise ValueError(
            "Invalid sampling frequency. Use 'daily', 'weekly', 'monthly', 'quarterly."
        )

    result = {}
    for ticker in tickers:
        if metric == "all":
            df_tick = df.loc[:, (slice(None), ticker)]
            df_tick.columns = df_tick.columns.droplevel("Ticker")
        else:
            df_tick = df.loc[:, ticker]
            df_tick = df_tick.to_frame(name=metric)
        df_tick = df_tick.reset_index()
        df_tick["Date"] = df_tick["Date"].dt.strftime("%Y-%m-%d")
        result[ticker] = df_tick.to_dict(orient="records")

    return result


@tool
def get_fred_data(
    series: str, start_date: str, end_date: str, sampling: str = "monthly"
) -> list[dict]:
    """Downloads data from the Federal Reserve Economic Data (FRED) database and returns it as dictionary.

    Examples:
        >>> get_fred_data("GDP", "2023-01-01", "2023-01-10")
        [{"Date": "2023-01-01", "GDP": 21.0}, {"Date": "2023-01-02", "GDP": 22.0}, ...]

    Args:
        series: The FRED series ID (e.g., "GDP").
        start_date: The start date for the data (e.g., "2023-01-01").
        end_date: The end date for the data in YYYY-MM-DD format (e.g., "2023-12-31").
        sampling: The frequency of the data. Can be "monthly", "quarterly", or "yearly". Defaults to "monthly".

    Returns:
        list: A list representing a list of dictionaries, where each dictionary contains 'Date' and the value of the FRED series.

    Raises:
        ValueError: If an invalid sampling frequency is provided.


    """
    df = pdr.data.DataReader(series, start=start_date, end=end_date, data_source="fred")

    if sampling == "monthly":
        df = df.resample("ME").last()
    elif sampling == "quarterly":
        df = df.resample("QE").last()
    elif sampling == "yearly":
        df = df.resample("YE").last()
    else:
        raise ValueError(
            "Invalid sampling frequency. Use 'monthly', 'quarterly', or 'yearly'."
        )

    df.reset_index(inplace=True)
    df["DATE"] = df["DATE"].dt.strftime("%Y-%m-%d")
    df.rename(columns={"DATE": "Date"}, inplace=True)

    result = df.to_dict(orient="records")

    return result


@tool
def run_ols_regression(y: List[float], X: List[float]) -> dict:
    """Runs a simple Ordinary Least Squares (OLS) regression.
    If you are using to compute beta, make sure the dates are aligned.

    Examples:
        >>> y = [1, 2, 3, 4, 5]
        >>> X = [2, 4, 5, 4, 5]
        >>> run_ols_regression(y, X)
        {"const": -0.4, "coef": 0.9}

    Args:
        y: The dependent variable.
        X: The independent variable(s).

    Returns:
        dict: A dictionary containing the constant and coefficient of the OLS regression.


    """
    X = np.array(X)
    y = np.array(y)
    X = sm.add_constant(X)
    model = sm.OLS(y, X)
    results = model.fit()
    params = results.params
    const, coef = params
    return {"const": const, "coef": coef}

In [8]:
# define the stock analysis agent
stock_analysis_agent = CodeAgent(
    tools=[get_ticker_data, get_fred_data, run_ols_regression, DuckDuckGoSearchTool()],
    model=model,
    name="stock_analyst_agent",
    description="A research agent that specializes in analyzing stock performance, computing technical indicators, and forecasting volatility.",
)

In [9]:
stock_analysis_agent.run("What immediate impact did Amazon's announcement of Alexa+ have on its stock price?")

╭───────────────────────────────────────── New run - stock_analyst_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ What immediate impact did Amazon's announcement of Alexa+ have on its stock price?                              │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0 ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  web_search_result = web_search(query="Amazon Alexa+ announcement date stock market impact")                      
  print(web_search_result)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Amazon Stock Gains On Alexa+ Plans. Here's What's New With The Gen AI 
...](https://www.investors.com/news/technology/amazon-stock-alexa-update-generative-ai/)
The rollout for Alexa+ will start with the U.S. in the "next few weeks," according to the company announcement. On 
the stock market today , Amazon stock gained a half-percent to close at 214.25 ...

[Amazon Announces AI Features For Alexa: Will It Finally Turn ... - 
Benzinga](https://www.benzinga.com/markets/equities/25/02/43990898/amazon-announces-ai-features-for-alexa-will-it-f
inally-turn-around-its-unprofitable-device-business)
What Happened: The Seattle-based company is rolling out an Alexa+ subscription, free for Amazon Prime subscribers, 
within the next few weeks. The subscription costs $19.99 per month for non-Prime ...

[Amazon Bets on Pricey AI Alexa as AI Battle 
Escalates](https://markets.businessinsider.com/news/stocks/amazon-bets-on-pricey-ai-alexa-as-ai-battle-escalates-10
34490296?op=1)
Discover stocks' market position with detailed competitor analyses. ... Amazon is bundling Alexa+ with its $139 
annual Prime membership, hoping to boost adoption. ... How This Could Impact Amazon ...

[Amazon (AMZN) Stock Ascends as it Says Hello to New AI-Enhanced 
Alexa](https://markets.businessinsider.com/news/stocks/amazon-amzn-stock-ascends-as-it-says-hello-to-new-ai-enhance
d-alexa-1034413653?op=1)
It is understood that 6 million people in the U.S. use an Alexa with the device holding a 66% market share. 
Worldwide, as of 2023 figures, it has sold 500 million.

[Amazon Stock Drifts Lower as Alexa+ Fails to Impress Investors - 
MSN](https://www.msn.com/en-us/money/topstocks/amazon-stock-drifts-lower-as-alexa-fails-to-impress-investors/ar-AA1
zYNhP)
Amazon's share price has declined more than 12% in the last week. Even Wednesday's Alexa+ launch failed to revive 
the struggling stock. Amazon's decline is part of a wider Big Tech pullback. While ...

[Amazon Stock Gains As Alexa Gets Long-Anticipated Generative AI ... - 
MSN](https://www.msn.com/en-us/news/technology/amazon-stock-gains-as-alexa-gets-long-anticipated-generative-ai-upda
te/ar-AA1zQDS5)
The rollout for Alexa+ will start with the U.S. in the "next few weeks," according to the company announcement. On 
the stock market today , Amazon stock gained a half-percent to close at 214.25 ...

[Is Amazon Stock a Buy, Sell, or Hold Ahead of Alexa+ Launch? - 
MSN](https://www.msn.com/en-us/news/technology/is-amazon-stock-a-buy-sell-or-hold-ahead-of-alexa-launch/ar-AA1AaMmw
)
With Alexa+ expected to drive even higher engagement and transaction volumes, Citi is keeping Amazon on its 
top-pick list, reaffirming its "Buy" rating and a $273 price target.

[Amazon.com, Inc. (AMZN): Alexa+ AI Expansion Signals Strong Growth 
...](https://finance.yahoo.com/news/amazon-com-inc-amzn-alexa-155353996.html)
We recently compiled a list of the 10 AI Stocks Making Waves on Wall Street. In this article, we are going to take 
a look at where Amazon.com, Inc. (NASDAQ:AMZN) stands against the other AI stocks.

[Amazon debuts gen AI-powered Alexa+ as it seeks to breathe life ... - 
AOL](https://www.aol.com/finance/amazon-debuts-gen-ai-powered-165348262.html)
Amazon stock was up about 2% following the announcement. Amazon is pouring billions of dollars into its generative 
AI effort as it seeks to ensure its place as a leader in the space.

[Everything you say to your Echo will be sent to Amazon starting on 
...](https://arstechnica.com/gadgets/2025/03/everything-you-say-to-your-echo-will-be-sent-to-amazon-starting-on-mar
ch-28/)
However, Amazon is betting big that Alexa+ can dig the voice assistant out of a financial pit. Amazon has publicly 
committed to keeping the free version of Alexa around, but Alexa+ is viewed as ...

Out: None

[Step 0: Duration 5.01 seconds| Input tokens: 3,433 | Output tokens: 79]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The Alexa+ announcement had an immediate positive impact on Amazon's stock price, with a 4.49%     
  increase over two trading days, rising from $167.08 to $174.58.")                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The Alexa+ announcement had an immediate positive impact on Amazon's stock price, with a 4.49% 
increase over two trading days, rising from $167.08 to $174.58.

[Step 3: Duration 3.56 seconds| Input tokens: 18,842 | Output tokens: 450]

"The Alexa+ announcement had an immediate positive impact on Amazon's stock price, with a 4.49% increase over two trading days, rising from $167.08 to $174.58."

In [10]:
stock_analysis_agent.run("How has Amazon's stock price changed since the Fed began lowering rates in 2024")

╭───────────────────────────────────────── New run - stock_analyst_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ How has Amazon's stock price changed since the Fed began lowering rates in 2024                                 │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0 ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  from datetime import datetime                                                                                    
                                                                                                                   
  # First, let's confirm the Fed's rate cut timing                                                                 
  search_result = web_search(query="When did Federal Reserve first lower interest rates in 2024")                  
  print(search_result)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Fed Cuts Interest Rates For First Time In 4 Years: Here's ... - 
Forbes](https://www.forbes.com/sites/dereksaul/2024/09/18/fed-cuts-interest-rates-for-first-time-in-4-years-heres-w
hat-it-means-for-you/)
It's the first cut to the federal funds rate since March 2020, bringing rates down to 4.75% to 5% from the 5.25% to
5.5% range they've sat since last July, the highest rates had been since 2001.

[Fed cuts rates by a half point at September 2024 
meeting](https://www.cnbc.com/2024/09/18/fed-cuts-rates-september-2024-.html)
The Federal Reserve on Wednesday lowered the federal funds rate to a range between 4.75%-5%.

[U.S. federal funds rate 1954-2025 | 
Statista](https://www.statista.com/statistics/187616/effective-rate-of-us-federal-funds-monthly/)
After maintaining historically low rates for nearly two years, the Federal Reserve began a series of rate hikes in 
early 2022, with the rate moving from 0.33 percent in April 2022 to 5.33 percent ...

[September 2024 Fed Meeting: Fed Cuts Rates by Half Point to Support 
...](https://www.jpmorgan.com/insights/outlook/economic-outlook/fed-meeting-september-2024)
At the September 2024 Federal Open Market Committee (FOMC) meeting, the Federal Reserve (Fed) lowered interest 
rates by 50 basis points, easing monetary policy for the first time in four years due to progress on the Fed's dual
mandate. This lowers the interest rate target to a range of 4.75% to 5%. 1

[The Fed Is Cutting Interest Rates: This Is What to Expect - U.S. News 
...](https://www.usnews.com/news/economy/articles/2024-09-18/the-fed-is-cutting-interest-rates-what-to-know)
Prior to its September meeting, the Fed was forecasting that the federal funds rate, used as a benchmark for bank 
to bank lending, would hit 5.1% this year, just a hair below the 5.25% to 5.50% ...

[Key takeaways from the Fed's decision to deliver a jumbo-sized interest 
...](https://www.cnn.com/2024/09/18/economy/interest-rate-cut-decision/index.html)
The Federal Reserve slashed interest rates aggressively Wednesday, announcing the first rate cut since March 2020. 
The half-point move paves the way for lower borrowing costs on everything from ...

[The Federal Reserve's first rate meeting of 2024 is today ... - CBS 
News](https://www.cbsnews.com/news/federal-reserve-interest-rate-cut-meetings-2024/)
When will the Fed cut interest rates? The Fed on Wednesday maintained the federal funds rate in a range of 5.25% to
5.5%. About 4 in 10 of economists polled by FactSet said they believe the first ...

[Fed Cuts Interest Rates for First Time in Four Years - The New York 
Times](https://www.nytimes.com/2024/09/18/business/economy/fed-reserve-interest-rate-cut.html)
Fed officials predicted that they would cut interest rates to 4.4 percent by the end of the year — much lower than 
the 5.1 percent they had been expecting in June, when they last released ...

[Fed to cut rates by 25 basis points on Sept. 18, twice more in 2024 
...](https://www.reuters.com/markets/us/fed-cut-rates-by-25-basis-points-sept-18-twice-more-2024-2024-09-10/)
The Federal Reserve will lower interest rates by 25 basis points at each of the U.S. central bank's three remaining
policy meetings in 2024, according to a majority of economists in a Reuters poll ...

[Fed cuts interest rates half a point, the first reduction since 2020 
...](https://www.latimes.com/business/story/2024-09-18/the-fed-lowers-interest-rates-for-first-time-since-2020-prom
ises-more-cuts-to-come)
Lower interest rates will ultimately help with credit card and real estate purchases, but it may take months for 
these benefits to seep to consumers. Fed cuts interest rates half a point, the ...

Out: None

[Step 0: Duration 4.48 seconds| Input tokens: 3,433 | Output tokens: 130]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Prepare the final analysis                                                                                     
  analysis = f"""                                                                                                  
  Amazon Stock Price Analysis Following Fed Rate Cut (September 2024):                                             
  - Initial Price: ${initial_price:.2f}                                                                            
  - Final Price: ${final_price:.2f}                                                                                
  - Percentage Increase: {percent_change:.2f}%                                                                     
                                                                                                                   
  Interpretation:                                                                                                  
  Amazon's stock price increased by {percent_change:.2f}% in the month following the Fed's first rate cut in       
  2024.                                                                                                            
  This positive movement suggests that the rate cut may have been perceived favorably by investors, potentially    
  due to expectations of improved economic conditions and lower borrowing costs.                                   
  """                                                                                                              
                                                                                                                   
  final_answer(analysis)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 
Amazon Stock Price Analysis Following Fed Rate Cut (September 2024):
- Initial Price: $176.25
- Final Price: $187.97000122070312
- Percentage Increase: 6.64964608266844%

Interpretation: 
Amazon's stock price increased by 6.64964608266844% in the month following the Fed's first rate cut in 2024. 
This positive movement suggests that the rate cut may have been perceived favorably by investors, potentially due 
to expectations of improved economic conditions and lower borrowing costs.

[Step 2: Duration 4.42 seconds| Input tokens: 13,590 | Output tokens: 608]

"\nAmazon Stock Price Analysis Following Fed Rate Cut (September 2024):\n- Initial Price: $176.25\n- Final Price: $187.97000122070312\n- Percentage Increase: 6.64964608266844%\n\nInterpretation: \nAmazon's stock price increased by 6.64964608266844% in the month following the Fed's first rate cut in 2024. \nThis positive movement suggests that the rate cut may have been perceived favorably by investors, potentially due to expectations of improved economic conditions and lower borrowing costs.\n"

In [11]:
stock_analysis_agent.run("What is the correlation between Amazon's stock price and the GDP of the United States?")

╭───────────────────────────────────────── New run - stock_analyst_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ What is the correlation between Amazon's stock price and the GDP of the United States?                          │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0 ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import statistics                                                                                                
                                                                                                                   
  # Get Amazon stock closing prices                                                                                
  amzn_data = get_ticker_data(                                                                                     
      tickers=["AMZN"],                                                                                            
      start_date="2010-01-01",                                                                                     
      end_date="2023-12-31",                                                                                       
      metric="Close",                                                                                              
      sampling="yearly"                                                                                            
  )                                                                                                                
                                                                                                                   
  # Get US GDP data                                                                                                
  gdp_data = get_fred_data(                                                                                        
      series="GDP",                                                                                                
      start_date="2010-01-01",                                                                                     
      end_date="2023-12-31",                                                                                       
      sampling="yearly"                                                                                            
  )                                                                                                                
                                                                                                                   
  print("Amazon Stock Data:", amzn_data)                                                                           
  print("GDP Data:", gdp_data)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

[*********************100%***********************]  1 of 1 completed


Code execution failed at line 'amzn_data = get_ticker_data(
    tickers=["AMZN"], 
    start_date="2010-01-01", 
    end_date="2023-12-31", 
    metric="Close", 
    sampling="yearly"
)' due to: ValueError: Invalid sampling frequency. Use 'daily', 'weekly', 'monthly', 'quarterly.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import math                                                                                                      
  import statistics                                                                                                
                                                                                                                   
  # Extract valid data points                                                                                      
  amzn_prices = [entry['Close'] for entry in amzn_data['AMZN'] if not math.isnan(entry['Close'])]                  
  gdp_values = [entry['GDP'] for entry in gdp_data if not math.isnan(entry['GDP'])]                                
                                                                                                                   
  # Ensure we have equal length data                                                                               
  min_length = min(len(amzn_prices), len(gdp_values))                                                              
  amzn_prices = amzn_prices[:min_length]                                                                           
  gdp_values = gdp_values[:min_length]                                                                             
                                                                                                                   
  # Compute correlation using OLS regression                                                                       
  correlation_result = run_ols_regression(y=amzn_prices, X=gdp_values)                                             
                                                                                                                   
  print("Correlation coefficient:", correlation_result['coef'])                                                    
  final_answer(correlation_result['coef'])                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Correlation coefficient: 0.0009147731342036517

Out - Final answer: 0.0009147731342036517

[Step 2: Duration 9.02 seconds| Input tokens: 19,041 | Output tokens: 746]

0.0009147731342036517

In [12]:
stock_analysis_agent.run("Compare and analyze the market beta for FAANG stocks since 2019")

╭───────────────────────────────────────── New run - stock_analyst_agent ─────────────────────────────────────────╮
│                                                                                                                 │
│ Compare and analyze the market beta for FAANG stocks since 2019                                                 │
│                                                                                                                 │
╰─ LiteLLMModel - bedrock/us.anthropic.claude-3-5-haiku-20241022-v1:0 ────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import statistics                                                                                                
  import math                                                                                                      
                                                                                                                   
  # Define FAANG stocks and market index                                                                           
  faang_stocks = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL']                                                         
  market_index = '^GSPC'  # S&P 500                                                                                
                                                                                                                   
  # Fetch stock data from 2019 onwards                                                                             
  start_date = '2019-01-01'                                                                                        
  end_date = '2023-12-31'                                                                                          
                                                                                                                   
  # Get market index data                                                                                          
  market_data = get_ticker_data([market_index], start_date, end_date, 'Close', 'monthly')                          
  market_prices = [entry['Close'] for entry in market_data[market_index]]                                          
  market_returns = [((market_prices[i] - market_prices[i-1]) / market_prices[i-1]) * 100 for i in range(1,         
  len(market_prices))]                                                                                             
                                                                                                                   
  # Calculate beta for each FAANG stock                                                                            
  betas = {}                                                                                                       
  for stock in faang_stocks:                                                                                       
      stock_data = get_ticker_data([stock], start_date, end_date, 'Close', 'monthly')                              
      stock_prices = [entry['Close'] for entry in stock_data[stock]]                                               
      stock_returns = [((stock_prices[i] - stock_prices[i-1]) / stock_prices[i-1]) * 100 for i in range(1,         
  len(stock_prices))]                                                                                              
                                                                                                                   
      # Run OLS regression to calculate beta                                                                       
      regression_result = run_ols_regression(stock_returns, market_returns)                                        
      betas[stock] = regression_result['coef']                                                                     
                                                                                                                   
  print("Market Betas for FAANG Stocks:")                                                                          
  for[38;2;248;248;242;

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Execution logs:
Market Betas for FAANG Stocks:
META: 1.1521975484519142
AAPL: 1.3144158931139294
AMZN: 1.1423087286946347
NFLX: 1.2155703090641943
GOOGL: 1.0584460910454017

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Prepare final analysis                                                                                         
  analysis = f"""FAANG Stocks Beta Analysis (2019-2023):                                                           
                                                                                                                   
  1. Beta Rankings (Most to Least Volatile):                                                                       
     - Apple (AAPL): {betas['AAPL']:.2f} - Highest market sensitivity                                              
     - Netflix (NFLX): {betas['NFLX']:.2f} - High market volatility                                                
     - Meta (META): {betas['META']:.2f} - Significant market correlation                                           
     - Amazon (AMZN): {betas['AMZN']:.2f} - Moderate market volatility                                             
     - Google (GOOGL): {betas['GOOGL']:.2f} - Closest to market movement                                           
                                                                                                                   
  2. Overall Insights:                                                                                             
     - Average FAANG Stock Beta: {avg_beta:.2f}                                                                    
     - All stocks show above-market volatility                                                                     
     - Indicates higher risk and potential for larger price swings                                                 
                                                                                                                   
  3. Interpretation:                                                                                               
     - Beta > 1 suggests stocks move more dramatically than the S&P 500                                            
     - Investors can expect amplified market movements for these stocks                                            
     - Potential for higher returns, but also higher potential losses                                              
  """                                                                                                              
                                                                                                                   
  final_answer(analysis)                                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: FAANG Stocks Beta Analysis (2019-2023):

1. Beta Rankings (Most to Least Volatile):
   - Apple (AAPL): 1.3144158931139294 - Highest market sensitivity
   - Netflix (NFLX): 1.2155703090641943 - High market volatility
   - Meta (META): 1.1521975484519142 - Significant market correlation
   - Amazon (AMZN): 1.1423087286946347 - Moderate market volatility
   - Google (GOOGL): 1.0584460910454017 - Closest to market movement

2. Overall Insights:
   - Average FAANG Stock Beta: 1.1765877140740149
   - All stocks show above-market volatility
   - Indicates higher risk and potential for larger price swings

3. Interpretation:
   - Beta > 1 suggests stocks move more dramatically than the S&P 500
   - Investors can expect amplified market movements for these stocks
   - Potential for higher returns, but also higher potential losses

[Step 2: Duration 7.36 seconds| Input tokens: 13,225 | Output tokens: 1,115]

'FAANG Stocks Beta Analysis (2019-2023):\n\n1. Beta Rankings (Most to Least Volatile):\n   - Apple (AAPL): 1.3144158931139294 - Highest market sensitivity\n   - Netflix (NFLX): 1.2155703090641943 - High market volatility\n   - Meta (META): 1.1521975484519142 - Significant market correlation\n   - Amazon (AMZN): 1.1423087286946347 - Moderate market volatility\n   - Google (GOOGL): 1.0584460910454017 - Closest to market movement\n\n2. Overall Insights:\n   - Average FAANG Stock Beta: 1.1765877140740149\n   - All stocks show above-market volatility\n   - Indicates higher risk and potential for larger price swings\n\n3. Interpretation:\n   - Beta > 1 suggests stocks move more dramatically than the S&P 500\n   - Investors can expect amplified market movements for these stocks\n   - Potential for higher returns, but also higher potential losses\n'